In [1]:
import numpy as np
import mne
from scipy import stats
from mne.stats import spatio_temporal_cluster_1samp_test, summarize_clusters_stc

In [2]:
def get_stcs(subjects_dir, fsave_vertices, directory, sub):
    src_fname = '/Applications/freesurfer/8.0.0/subjects/fsaverage/bem/fsaverage-ico-4-src.fif'
    src = mne.read_source_spaces(src_fname)

    stc_prod_ident = mne.read_source_estimate(directory + sub + '_ident_prod-lh.stc', subject=sub)
    stc_prod_unrel = mne.read_source_estimate(directory + sub + '_unrel_prod-lh.stc', subject=sub)
    stc_comp_ident = mne.read_source_estimate(directory + sub + '_ident_comp-lh.stc', subject=sub)
    stc_comp_unrel = mne.read_source_estimate(directory + sub + '_unrel_comp-lh.stc', subject=sub)

    morph1 = mne.compute_source_morph(
        stc_prod_ident,
        subject_from=sub,
        subject_to='fsaverage',
        src_to=src,
        subjects_dir=subjects_dir)
    morph2 = mne.compute_source_morph(
        stc_prod_unrel,
        subject_from=sub,
        subject_to='fsaverage',
        src_to = src,
        subjects_dir=subjects_dir)
    morph3 = mne.compute_source_morph(
        stc_comp_ident,
        subject_from=sub,
        subject_to='fsaverage',
        src_to = src,
        subjects_dir=subjects_dir)
    morph4 = mne.compute_source_morph(
        stc_comp_unrel,
        subject_from=sub,
        subject_to='fsaverage',
        src_to = src,
        subjects_dir=subjects_dir)

    prod_ident = morph1.apply(stc_prod_ident)
    prod_unrel = morph2.apply(stc_prod_unrel)
    comp_ident = morph3.apply(stc_comp_ident)
    comp_unrel = morph4.apply(stc_comp_unrel)

    return prod_ident, prod_unrel, comp_ident, comp_unrel

In [3]:
subs = ['R3250', 'R3254', 'R3261', 'R3264', 'R3270','R3271','R3272','R3273','R3275','R3277','R3279','R3285','R3286','R3289','R3290','R3285','R3286','R3289','R3290','R3326','R3327','R3329']
n_subjects = len(subs)
subjects_dir = '/Applications/freesurfer/8.0.0/subjects'
src_fname = '/Applications/freesurfer/8.0.0/subjects/fsaverage/bem/fsaverage-ico-4-src.fif'

ident_prod_stcs = []
unrel_prod_stcs = []
ident_comp_stcs = []
unrel_comp_stcs = []

src = mne.read_source_spaces(src_fname)

fsave_vertices = [s["vertno"] for s in src]


for sub in subs:
    directory = f'/Users/audreylaun/Library/CloudStorage/Box-Box/Starling/Experiment1/MEG_data/Testing/{sub}/'
    stc_prod_ident_fsavg, stc_prod_unrel_fsavg, stc_comp_ident_fsavg, stc_comp_unrel_fsavg = get_stcs(
        subjects_dir, fsave_vertices, directory, sub)
    tstep = stc_prod_ident_fsavg.tstep * 1000  # ms

    ident_prod_stcs.append(stc_prod_ident_fsavg)
    unrel_prod_stcs.append(stc_prod_unrel_fsavg)
    ident_comp_stcs.append(stc_comp_ident_fsavg)
    unrel_comp_stcs.append(stc_comp_unrel_fsavg)

comp_ident_data = np.stack([x.data for x in ident_comp_stcs], axis=0)  # shape: subjects x vertices x times
comp_unrel_data = np.stack([x.data for x in unrel_comp_stcs], axis=0)

# Stack conditions, reorder axes: subjects, times, vertices, conditions
X = np.stack([comp_ident_data, comp_unrel_data])  # (2 conditions, subjects, vertices, times)
X = np.transpose(X, (1, 3, 2, 0))  # now: subjects x times x vertices x conditions

# Use magnitude and compute difference between conditions
X = np.abs(X)
X = X[:, :, :, 1] - X[:, :, :, 0]  # difference: condition 1 - 0

# Select time points between 200 and 600 ms
times = ident_prod_stcs[0].times * 1000  # convert to ms
time_mask = (times >= 350) & (times <= 450)
X = X[:, time_mask, :]  # subjects x selected times x all vertices

# Restrict to left hemisphere vertices only
n_lh = len(fsave_vertices[0])  # 2562
n_rh = len(fsave_vertices[1])  # 2562

# Data currently has vertices for both hemispheres concatenated along last axis: 5124 vertices
# Separate hemispheres:
X_lh = X[:, :, :n_lh]  # left hemisphere data

# Prepare adjacency matrix
adjacency = mne.spatial_src_adjacency(src)  # full adjacency 5124x5124
adjacency = adjacency.tocsr()
adj_lh = adjacency[:n_lh, :n_lh]  # left hemisphere adjacency

    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
surface source space present ...
Computing morph matrix...
    Left-hemisphere map read.
    Right-hemisphere map read.
    5 smooth iterations done.
    5 smooth iterations done.
[done]
[done]
surface source space present ...
Computing morph matrix...
    Left-hemisphere map read.
    Right-hemisphere map read.
    5 smooth iterations done.
    5 smooth iterations done.
[done]
[done]
surface source space present ...
Computing morph matrix...
    Left-hemisphere map read.
    Right-hemisphere map read.
    5 smooth iterations done.
    5 smooth iterations done.
[done]
[done]
surface source space present ...
Computing morph matrix...
    Left-hemisphere map read.
    Right-hemisphere map read.
    5 smooth iterations done.
    5 smooth iterations done.
[done]
[done]
    Readin

In [4]:
# Cluster forming threshold
p_threshold = 0.05
df = n_subjects - 1
t_threshold = stats.distributions.t.ppf(1 - p_threshold / 2, df=df)

print("Clustering.")
T_obs, clusters, cluster_p_values, H0 = clu = spatio_temporal_cluster_1samp_test(
    X_lh,
    adjacency=adj_lh,
    n_jobs=None,
    threshold=t_threshold,
    buffer_size=None,
    verbose=True,
)

Clustering.
stat_fun(H1): min=-7.008859634399414 max=6.509446620941162
Running initial clustering …
Found 752 clusters


  0%|          | Permuting : 0/1023 [00:00<?,       ?it/s]

In [5]:
stc_all_cluster_vis = summarize_clusters_stc(
    clu, tstep=tstep, p_thresh=1, vertices=[src[0]['vertno']], subject="fsaverage")